<a href="https://colab.research.google.com/github/dantae74/tensorflow/blob/main/01-basic-sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Tensorflow 2.0

In [ ]:
import os
import tarfile
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
DOWNLOAD_ROOT = 'https://raw.githubusercontent.com/ageron/handson-ml/master/'
HOUSING_PATH = os.path.join('datasets', 'housing')
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

In [ ]:
def fetch_housing_data(housing_url = HOUSING_URL, housing_path = HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, 'housing.tgz')
    urllib.request.urlretrieve(housing_url, tgz_path)

    housig_tgz = tarfile.open(tgz_path)
    housig_tgz.extractall(path=housing_path)
    housig_tgz.close()

fetch_housing_data()

In [ ]:
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, 'housing.csv')
    return pd.read_csv(csv_path)

housing = load_housing_data()

In [ ]:
housing.head()

In [ ]:
housing.info()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

In [ ]:
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
housing['income_cat'] = pd.cut(housing['median_income'],
                               bins=[0.,1.5,3.0,4.5,6., np.inf],
                               labels=[1,2,3,4,5])

In [ ]:
housing['income_cat'].hist()

### 소득 카테고리 기반 계층 샘플링

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
strat_test_set['income_cat'].value_counts() / len(strat_test_set)

In [ ]:
# income_cat 특성 삭제
for set_ in (strat_test_set, strat_train_set):
  set_.drop("income_cat", axis=1, inplace=True)


In [ ]:
# 훈련세트를 보존하기 위한 복사본 만들기
housing = strat_train_set.copy()

In [ ]:
# 지리정보(위도와 경도)를 사용한 데이터 시각화
housing.plot(kind='scatter', x="longitude", y="latitude")

In [ ]:
# 캘리포니아 지역의 특별한 패턴 정보 찾기 (밀집지역 부각한 산점도)
housing.plot(kind='scatter', x="longitude", y="latitude", alpha=0.1)

In [ ]:
# 캘리포니아 주택가격(빨간색은 높은가격, 파란색은 낮은가격), 큰원은 인구가 밀집된 지역
housing.plot(kind='scatter', x="longitude", y="latitude", alpha=0.4,
             s=housing['population']/100, label='population', figsize=(10,7),
             c='median_house_value', cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)
plt.legend()

In [ ]:
# 상관관계 조사
corr_matrix = housing.corr()

corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
housing.columns

In [ ]:
# 상관관계 (산점도)
from pandas.plotting import scatter_matrix

attributes = ['median_house_value', 'median_income', 'housing_median_age', 'total_bedrooms']
scatter_matrix(housing[attributes], figsize=(12,8))

In [ ]:
# median_house_value에 대한 median_income 상관관계 산점도
housing.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.1)

In [ ]:
# 특성 조합 특성 조합
housing['rooms_per_household'] = housing['total_bedrooms'] / housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
housing['population_per_household'] = housing['population'] / housing['households']
housing['rooms_per_household'] = housing['total_bedrooms'] / housing['households']
housing['bedrooms_per_room'] = housing['total_bedrooms'] / housing['total_rooms']
housing['population_per_household'] = housing['population'] / housing['households']

In [ ]:
corr_matrix = housing.corr()
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
# 예측변수와 타깃값 분리

housing = strat_train_set.drop('median_house_value', axis=1)
housing_labels = strat_train_set['median_house_value'].copy()

In [ ]:
# 데이터 정제

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

In [ ]:
# 수치형 특성에 누락된 값에 대해 중앙값 적용
housing_num = housing.drop('ocean_proximity', axis=1)

In [ ]:
imputer.fit(housing_num)

In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

In [ ]:
# 훈련 세트에 누락된 값을 학습한 중앙값으로 변경
# 결과로 numpy의 arrary인 type을 다시 pandas의 dataframe으로 변환
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [ ]:
housing_cat = housing[['ocean_proximity']]
housing_cat.head()

In [ ]:
# 범주형 데이터 수치형으로 변환
# Ordinal Encoder의 경우  머신러닝이 가까이 있는 두값이 떨어져 있는 두값보다 더 비슷하다고 생각
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[0:10]

In [ ]:
ordinal_encoder.categories_

In [ ]:
# One-Hot Encoding
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
housing_cat_1hot.toarray()

In [ ]:
cat_encoder.categories_

In [ ]:
# 나만의 변환기
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributeAdder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedrooms_per_room = True): 
    self.add_bedrooms_per_room = add_bedrooms_per_room
  def fit(self, X, y=None):
    return self
  def transform(self, X):
    rooms_per_household = X[:,rooms_ix] / X[:,household_ix]
    population_per_household = X[:,population_ix] / X[:,household_ix]
    if self.add_bedrooms_per_room:
      bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
    else:
      return np.c_[X, rooms_per_household, population_per_household]
  
attr_adder = CombinedAttributeAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
# 변환 Pipeline
# fit_transform() 메소드가 있어야 함.

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='median')),
                         ('attribs_adder', CombinedAttributeAdder()),
                         ('std_scaler', StandardScaler)
          ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
# 변환 Pipeline
# fit_transform() 메소드가 있어야 함.

from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ['ocean_proximity']

full_pipline = ColumnTransformer([
                         ('num', num_pipeline, num_attribs),
                         ('cat', OneHotEncoder(), cat_attribs)
          ])

housing_prepared = full_pipline.fit_transform(housing)